In [13]:
# -*- coding: utf8 -*-

"""Contains function to perform GX-FBA"""

from __future__ import absolute_import

import six
import numpy as np
from optlang.symbolics import Zero
from cobra.flux_analysis import flux_variability_analysis as fva


def gx_fba(model, expression_profile, conditon=None, reg_threshold=0.5, flux_threshold=100.0):
    """
    GX-FBA[1]
    
    Parameters
    ----------
    model: cobra.Model
        The constraint-based model to perform GX-FBA on.
    expression_profile: ExpressionProfile
        The expression profile to integrate in the model.
    condition: str or int, optional
        The condition from the expression profile.
    reg_threshold: float, optional (default 0.5)
        The minumum fold change to be considered for up or down regulation.
    flux_threshold: float, optional (default 100.0)
        The maximum flux value to be considered for a reaction. If not obeyed
        by reaction, it is discarded from analysis.
    
    Returns
    -------
    model, cobra.Solution
    
    References
    ----------
    .. [1] Navid and Almaas (2012).
           Genome-level transcription data of Yersinia pestis analyzed with a New metabolic constraint-based approach.
           BMC Systems Biology 2012 6:150.
           doi:10.1186/1752-0509-6-150
    """
    
    assert isinstance(model, cobra.Model)
    assert isinstance(expression_profile, ExpressionProfile)
    assert isinstance(condition, (str, int, None))
    assert isinstance(reg_threshold, float)
    assert isinstance(flux_threshold, float)

    with model:
        prob = model.problem
        # Changing the LP solution method to Interior Point
        model.solver.configuration.lp_method = "barrier"
        # Wild-type flux distribution
        sol_wt = model.optimize()
        # Flux variability analysis
        #TODO: Make minimal flux of biomass to zero for calculation
        fva_sol = fva(model)
        # Mean possible flux for each reaction
        fva_mean = fva_sol.mean(axis=1)
        # Avg. flux carried by active reactions
        avg_flux = fva_mean.mean()
        
        #TODO: Convert incoming expression profile conditions to log2(exp)
        condition = expression_profile.conditions[0] if condition is None else condition
        reaction_profile = expression_profile.to_reaction_dict(condition, model, not_measured_value, normalization)
        
        obj_vars = []
        for rxn_id, rxn_exp in six.iteritems(reaction_profile):
            rxn = model.reactions.get_by_id(rxn_id)
            if rxn not in model.exchanges and rxn.reversibility is False \
                and fva_mean[rxn_id] > flux_threshold:
                if rxn_exp > 0.0:
                    rxn.upper_bound = rxn_exp * rxn.flux_expression
                elif rxn_exp < 0.0:
                    rxn.lower_bound = rxn_exp * rxn.flux_expression
                
                obj_var = prob.Variable(rxn_exp * (rxn.flux_expression / fva_mean[rxn_id]))
                obj_vars.append(obj_var)
                
        model.add_cons_vars(obj_vars)
        model.objective = prob.Objective(Zero, direction="max", sloppy=True)
        model.objective.set_linear_coefficients({v: 1.0 for v in obj_vars})
        
        solution = model.optimize()
        
        return (model, solution)

In [1]:
import cobra.test

In [2]:
import cobra

In [3]:
model = cobra.test.create_test_model("textbook")

In [4]:
sol_wt = model.optimize()

In [5]:
fva_sol = cobra.flux_analysis.flux_variability_analysis(model)

In [7]:
model.objective.expression

1.0*Biomass_Ecoli_core - 1.0*Biomass_Ecoli_core_reverse_2cdba

In [17]:
fva_sol.maximum["Biomass_Ecoli_core"]

0.8739215069684307

In [18]:
model.reactions.Biomass_Ecoli_core

Reaction identifier,Biomass_Ecoli_core
Name,Biomass Objective Function with GAM
Memory address,0x0118d79710
Stoichiometry,1.496 3pg_c + 3.7478 accoa_c + 59.81 atp_c + 0.361 e4p_c + 0.0709 f6p_c + 0.129 g3p_c + 0.205 g6p_c + 0.2557 gln__L_c + 4.9414 glu__L_c + 59.81 h2o_c + 3.547 nad_c + 13.0279 nadph_c + 1.7867 oaa_c ... 1.496 3-Phospho-D-glycerate + 3.7478 Acetyl-CoA + 59.81 ATP + 0.361 D-Erythrose 4-phosphate + 0.0709 D-Fructose 6-phosphate + 0.129 Glyceraldehyde 3-phosphate + 0.205 D-Glucose 6-phosphate + 0.2557...
GPR,
Lower bound,0.0
Upper bound,1000.0


In [7]:
sol_wt

,fluxes,reduced_costs
ACALD,0.000000,6.938894e-18
ACALDt,0.000000,0.000000e+00
ACKr,0.000000,1.040834e-17
ACONTa,6.007250,0.000000e+00
ACONTb,6.007250,1.387779e-17
...,...,...
TALA,1.496984,-2.775558e-17
THD2,0.000000,-2.546243e-03
TKT1,1.496984,-1.387779e-17
TKT2,1.181498,1.387779e-17


In [8]:
fva_mean = fva_sol.mean(axis=1)

In [9]:
avg_flux = fva_mean.mean()

In [10]:
fva_mean["ACALD"]

4.844711052085257e-15

In [11]:
import numpy as np

In [18]:
sol_wt.fluxes[sol_wt.fluxes == 0.0]

ACALD          0.0
ACALDt         0.0
ACKr           0.0
ACt2r          0.0
ADK1           0.0
AKGt2r         0.0
ALCD2x         0.0
D_LACt2        0.0
ETOHt2r        0.0
EX_ac_e        0.0
EX_acald_e     0.0
EX_akg_e       0.0
EX_etoh_e      0.0
EX_for_e       0.0
EX_fru_e       0.0
EX_fum_e       0.0
EX_gln__L_e    0.0
EX_glu__L_e    0.0
EX_lac__D_e    0.0
EX_mal__L_e    0.0
EX_pyr_e       0.0
EX_succ_e      0.0
FBP            0.0
FORt2          0.0
FORti          0.0
FRD7           0.0
FRUpts2        0.0
FUMt2_2        0.0
GLUN           0.0
GLUSy          0.0
GLUt2r         0.0
ICL            0.0
LDH_D          0.0
MALS           0.0
MALt2_2        0.0
ME1            0.0
ME2            0.0
NADTRHD        0.0
PFL            0.0
PPCK           0.0
PPS            0.0
PTAr           0.0
PYRt2          0.0
SUCCt2_2       0.0
SUCCt3         0.0
THD2           0.0
Name: fluxes, dtype: float64